## Module 10 Class activities
This notebook is a starting point for the exercises and activities that we'll do in class. 

We'll revisit our analysis of ADUs, and try and speed things up and reduce memory usage.

Then, we'll have some practice with SQL.

Before you attempt any of these activities, make sure to watch the video lectures for this module.

### Optimizing data types and parsing csv files
Let's revist the ADU data that we saw in Lecture 12 (classification). There are two csv files that we read in – one with the permit data, and one with the parcel data.

In [ ]:
import pandas as pd
permits = pd.read_csv('../lectures/data/ADUs/ADU_permits.csv')  # this file should be in your GitHub folder
permits.info() 

In [ ]:
parcels = pd.read_csv('../lectures/data/ADUS/parcels.csv')
parcels.info()

For me, the permits data takes 492KB of memory. The parcels data uses 84MB. Imagine that you had a much bigger data file (perhaps more counties, or perhaps more columns). How can you reduce the memory requirements of these dataframes?

*Hint*: I didn't mention this in the lecture, but pandas has a new "nullable integer" datatype which can take missing values (NaNs).

To take advantage of this, use `Int16`, `Int32`, etc. rather than `int16`, `int32`, etc. The capital I is the nullable integer datatype; the lower-case i is the numpy datatype. [See here for a detailed explanation.](https://pandas.pydata.org/docs/user_guide/integer_na.html)

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Reduce the memory usage of each dataframe as much as you can.
</div>

In [ ]:
# your code here

# look at the dataframe
# it seems that parcel is an object; the others are float64
print(permits.info())

# the maximum of those thre columns is 7563
print(permits.describe())

# it looks like they are integers and would fit into int16 but let's check
# one way to do this is to see if the sum of each column is the same
for col in ['Assessor Book', 'Assessor Page', '# of Accessory Dwelling Units']:
    assert permits[col].sum() == permits[col].astype('int16').sum()
    # if we didn't get an error on the previous line, we are good to continue
    permits[col] = permits[col].astype('int16')

# we reduced memory consumption by more than half!
print(permits.info())

In [ ]:
# now let's do the same for the parcels
print(parcels.info())

for col in parcels.columns:
    if parcels[col].dtypes=='object': 
        print ('Skipping {} because it is an object'.format(col))
    elif col in ['CENTER_LAT', 'CENTER_LON']:
        print('Skipping lat/lon columns')
    else:
        # when I first ran this, I got an error on SQFTmain1
        # that has larger numbers than will fit into Int16
        # so let's try it with Int16, otherwise use Int32
        try:
            parcels[col] = parcels[col].astype('Int16')
        except: # we got an error
            parcels[col] = parcels[col].astype('Int32')

        
print(parcels.info())

Now, imagine that the dataframes wouldn't even load into memory in the first place?

Also imagine that you don't need the `Bathrooms1` column.

Use additional arguments to `pd.read_csv()` and the datatypes you used in the answer above to load in the dataframes again.

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Load the dataframes from disk in the most memory-efficient way possible.
</div>

In [ ]:
# your code here

permits = pd.read_csv('../lectures/data/ADUs/ADU_permits.csv', dtype={'Assessor Book':'int16', 
            'Assessor Page':'int16', '# of Accessory Dwelling Units':'int16'})
print(permits.info())

# since we already read in parcels, we can access its columns, and then drop Bathrooms1
cols = [col for col in parcels.columns if col!='Bathrooms1']

parcels = pd.read_csv('../lectures/data/ADUs/parcels.csv', dtype={'YearBuilt1':'Int16', 'Units1':'Int16',
        'Bedrooms1':'Int16', 'Bathrooms1':'Int16', 'SQFTmain1':'Int32', 
        'Roll_LandValue':'Int32', 'Roll_ImpValue':'Int32',
        'Roll_LandBaseYear':'Int32', 'Roll_ImpBaseYear':'Int32'}, 
        usecols=cols)

print(parcels.info())

### Speeding up joins

Now let's try and join them together in an efficient way.

Remember `%timeit` (for a line of code) and `%%timeit` (for a cell) will tell you how long your code takes to run.

Before we join, we need to drop the rows with no parcel number, and create a concatenated column for the APN.

In [ ]:
permits = permits[permits['Assessor Parcel']!='***']
permits['APN'] = (permits['Assessor Book'].astype(int).astype(str).str.zfill(4) + '-' 
                   + permits['Assessor Page'].astype(int).astype(str).str.zfill(3) + '-'
                   + permits['Assessor Parcel'].astype(int).astype(str).str.zfill(3))
#drop the duplicates (take the first)
permits = permits.groupby('APN').first()
parcels = parcels.groupby('APN').first()

permits.head()

This is how we joined the parcels in lecture.

In [ ]:
joinedDf = parcels.join(permits, how='left') # left is the default so we could omit that argument

How long does that take?

In [ ]:
%timeit joinedDf = parcels.join(permits, how='left')

What if we use `pd.merge()` on a column, rather than the index? 

Let's first reset the indexes, so that `APN` becomes a regular column.

In [ ]:
parcels.reset_index(inplace=True)
permits.reset_index(inplace=True)

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> How long does it take to merge using the columns?</div>

In [ ]:
# your code here
%timeit joinedDf = pd.merge(parcels, permits, on='APN')

If you are curious, indexes are faster because they are [stored as hashes](https://stackoverflow.com/questions/27238066/what-is-the-point-of-indexing-in-pandas).

Before we move on, let's create the `has_adu` column as we did before.

In [ ]:
joinedDf['has_adu'] = joinedDf['# of Accessory Dwelling Units']>=1

## SQL

Finally, let's do some simple SQL queries on the joined dataframe.

Implement the following queries as SQL, using `pandasql`.

1. What's the total land value of all the parcels? (The column `Roll_LandValue`.)

2. How many parcels have ADUs? 

3. What's the average area of the building size (`SQFTmain1`), with and without an ADU? (Hint: use `GROUP BY`.)

4. Same as above, but for residential parcels only (`UseType='Residential'`)

Compare your results to a typical `pandas` query.

In [ ]:
from pandasql import PandaSQL
pdsql = PandaSQL()

# your code here

print(pdsql('SELECT SUM(Roll_LandValue) FROM joinedDf;'))
print(joinedDf.Roll_LandValue.sum())

In [ ]:
print(pdsql('SELECT COUNT(*) FROM joinedDf WHERE has_adu;'))
print(joinedDf.has_adu.sum())

In [ ]:
print(pdsql('SELECT AVG(SQFTmain1) FROM joinedDf GROUP BY has_adu;'))
print(joinedDf.groupby('has_adu').SQFTmain1.mean())

In [ ]:
print(pdsql("SELECT AVG(SQFTmain1) FROM joinedDf WHERE UseType='Residential' GROUP BY has_adu;"))
print(joinedDf[joinedDf.UseType=='Residential'].groupby('has_adu').SQFTmain1.mean())

<div class="alert alert-block alert-info">
<h3>You should now be able to:</h3>
<ul>
  <li>Deal with larger datasets on a standard laptop computer</li>
  <li>Profile your code to identify bottlenecks</li>
  <li>Write simple SQL queries</li>
  <li>Go forth and do data science!</li>
</ul>
</div>